In [188]:
import sys
code_path = '../../network_bio_toolkit'
sys.path.append(code_path)

import importlib
import Heat3
importlib.reload(Heat3)

import create_graph
importlib.reload(create_graph)

from gprofiler import GProfiler
import networkx as nx
import pandas as pd
import community
import heat_and_cluster

import visJS2jupyter

import visJS2jupyter.visualizations as visualizations
importlib.reload(visualizations)

import visJS2jupyter.visJS_module as visJS_module
importlib.reload(visJS_module)

import matplotlib.pyplot as plt
import numpy as np

In [189]:
symbol = 'symbol'
entrez = 'entrez'

human = 'human'
mouse = 'mouse'

heat = Heat3.Heat(gene_type = symbol, species = mouse)

In [190]:
#filename = '../../background_networks/10090.protein.links.v10.5.txt'
#heat.load_STRING_links(filename, confidence_filter = 700)

#print('\nNumber of interactions: ' + str(len(list(heat.DG_universe.edges()))))

In [191]:
# load DEG file
DEG_filename = '../../DEG_databases/DE_CoeffspaceFlight - groundControl_glds48_20180312.csv'  
heat.create_DEG_list(DEG_filename, p_value_filter = 0.02, sep = ',')

print('Number of DEG\'s: ' + str(len(heat.DEG_list)))

Number of DEG's: 45


../../network_bio_toolkit\create_graph.py:277: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  df = pd.DataFrame.from_csv(filename, sep=sep)


In [192]:
#saved_DG_universe = heat.DG_universe
heat.DG_universe = saved_DG_universe

In [193]:
#heat.normalized_adj_matrix()

In [194]:
#saved_Wprime = heat.Wprime
heat.Wprime = saved_Wprime

In [130]:
heat.draw_heat_prop(num_nodes = 500,
#                  random_walk = False,
                  edge_width = 2,
                  edge_smooth_enabled = True,
                  edge_smooth_type = 'bezier',
                  node_size_multiplier = 5,
                  hover = False,
                  hover_connected_edges = False,
                  largest_connected_component = True,
                  physics_enabled = True,
                  node_font_size = 40,
                  graph_id = 1,
                  node_shadow_x = 6)

In [131]:
heat.draw_clustering(rad_positions = False,
                k = None,
                largest_connected_component = True,
                num_top_genes = 500,
                cluster_size_cut_off = 5,
                remove_stray_nodes = True,
                node_spacing = 700,
                node_size_multiplier = 10,
                physics_enabled = True,
                node_font_size = 40,
                graph_id = 2,
                edge_width = 2,
                edge_smooth_enabled = True,
                edge_smooth_type = 'bezier',
                hover = False,
                hover_connected_edges = False
               )

In [132]:
heat.draw_clustering(num_top_genes = 500,
                cluster_size_cut_off = 5,
                remove_stray_nodes = True,
                r = 0.4,
                node_spacing = 600,
                node_size_multiplier = 4,
                physics_enabled = False,
                node_font_size = 15,
                graph_id = 3,
                node_shadow_x = 6,
                largest_connected_component = True,
                color_lfc = True,
                vmin = -5,
                vmax = 5
               )

In [195]:
heat.draw_clustering_with_annotation(graph_id = 4,
                        num_nodes = 500,
                        annotation = True,
                        node_spacing = 700,
                        node_size_multiplier = 5,
                        physics_enabled = False,
                        node_font_size = 20,
                        color_lfc = True,
                        node_size = 7)

Annotating cluster 0 of 9...
Annotating cluster 1 of 9...
Annotating cluster 2 of 9...
Annotating cluster 3 of 9...
Annotating cluster 4 of 9...
Annotating cluster 5 of 9...
Annotating cluster 6 of 9...
Annotating cluster 7 of 9...
Annotating cluster 8 of 9...
Annotating cluster 9 of 9...
Done!
